# Homework 10: Cleaning data with Regular Expressions

Time to use regular expressions!

# Hints and notes

### Opening files in subdirectories

Notice that this notebook might be **homework/**, but!!! the csvs and text files might be in **homework/scraped/** or **/homework/scraped/minutes_pdfs** or **/homework/pdfs/**. To open a file in a subdirectory, instead of having the filename be `"file.csv"` you'll just use `"some/subfolder/file.csv"`

### Opening text files

This will open up a file, read it in and show you the first 500 characters.

```python
contents = open("your-filename.txt").read()
contents[0:500]
```

> You might need `open("your-filename.txt", encoding="utf8").read()`

### Using regex

For some dumb reason you need to put `r` in front of the string you use when you're talking about regex. Just plain `"(\d\d\d)"` will usually work, but *sometimes* it won't and you'll need `r"(\d\d\d)`. It's best to just use the `r` all of the time, if you can remember!

### Using `.str.extract`

When you use `.str.extract`, you're always going to **capture one thing** and save it to a new column. You need to wrap the things you're interested in with parenthesis `(` `)`.

```python
df['phone_number'] = df['old_column'].str.extract(r"My phone number is (\d\d\d-\d\d\d-\d\d\d\d)")
```

### Setting pandas options

Pandas has a lot of options, like how many columns or rows it will show you, or how many characters it will show in a column before it stops showing you anything. Here are a few useful ones:

* `display.max_cols`: Number of columns to show at once
* `display.max_rows`: Number of rows to show at once
* `display.max_colwidth`: Maximum number of characters displayed from a string

You can set them using `pd.set_option("display.max_rows", 1000)`, for example, to show 1000 rows at a time. You can find a lot more at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.set_option.html

### Regular expressions reference

I personally think http://www.regular-expressions.info/ is a wonderful wonderful reference (and tutorial), even if it's ugly! But here's a quick reference for you:

* `\d` is a digit
* `\d*` is zero or more digits 
* `\d+` is one or more digits
* `.` matches anything for ONE character
* `.*` is "give me anything forever"
* `\s` is whitespace, a.k.a. spaces and tabs
* `\w` is a word character, which includes capital and lowercase letters, numbers and hyphens.
* You can put `*` after anything, so `\w*` would mean "as many word characters as you can find"
* `\b` is a word boundary (you'll need the `r""` thing for this one)
* `( )` is a "capture group" for saving something
* `\1` is used when doing find/replace to say "put the first captured group here" (note, it's a dollar sign instead of a backslash in some editors)
* `[ABCDE]` is a character class, which means "match one of these, I don't care which"
* dollar sign means "end of the line"
* caret ^ means "beginning of the line"
* `\.` means "no really seriously I mean a period not just anything"
* You can use `\` with anything else that would normally be a special character, too, not just periods. `(` or `[` or whatever.

### Cleaning up extracted columns

Sometimes you get `\n` (newlines) or spaces or `\t` (tabs) or stuff at the beginning or the end of your column. `.str.strip()` will usually take care of that, just attach it after your `.str.extract()`

After you extract something, it's still a string even though you look at it and know it's a number. Use `.astype(int)` to turn it into an integer (no decimal) or `.astype(float)` to turn it into a float (yes decimal)

### Writing regular expressions in general

Even if I'm using regex in pandas or Python, I like to test them in my text editor with "Find." The highlighting really helps me see if I'm matching things! I also like to think "what stays the same?" when designing patterns, write those parts first, then fill in the blanks with what I want to capture.

## Importing

There might be more, I just wanted to put this up here for the `pd.set_option` part. It allows you to see a lot of content in a single column of pandas, which will be important for some parts below.

In [1]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', 500)

# Part 1: Using `.str.extract` to pull data from columns in pandas

## 1.1 H&M

Open up `hm.csv` from the `scraped` directory. I want **four new columns**:

1. `price_original`, the original price, one of the new price
2. `price_discounted`, the discounted price
3. `pct_discount`, the percent discount
4. `article_id`, the article id (from the url)

Save as **hm_cleaned.csv**.

**Note:** When you look at it, it... won't look right. I don't know why, pandas is weird. Look at the `price` column by itself using `df['price']` before you write your regex.

**Tip:** Remember that `$` is a special regex symbol! You might need to escape it.

**Tip:** When doing `.str.extract`, the whole match doesn't get captured, only what you put `()` around! Think about anchoring to different points of the string, or things in the string.

**Tip:** Not all prices have cents!

**Tip:** Your first instinct about how to compute the percent discount is probably wrong

In [2]:
df = pd.read_csv("scraped/hm.csv")
df.head()

,name,price,url
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A


In [3]:
df['price_original'] = df.price.str.extract("( \$\d.*)")
# df['price_original'] = df.price.str.extract("( [$]d.*)")
#df.head()

In [4]:
df['price_discounted'] = df.price.str.extract("(\$\d.* )")
df.head()

,name,price,url,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,$129,$59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,$17.99,$6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,$17.99,$7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,$12.99,$6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,$24.99,$17.99


In [5]:
#get rid of the dollar signs in the discounted/original columns
df['price_discounted'] = df['price_discounted'].str.replace(r'([$])','')
df['price_original'] = df['price_original'].str.replace(r'([$])','')
df.head()

,name,price,url,price_original,price_discounted
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129,59.99
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99


In [6]:
#convert both columns to floats
df['price_discounted'] = df['price_discounted'].astype(float)
df['price_original'] = df['price_original'].astype(float)

df['pct_discount'] = 100- round((df['price_discounted'] / df['price_original']*100))
df.head()

,name,price,url,price_original,price_discounted,pct_discount
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.0
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.0
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,56.0
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.0
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0


In [16]:
df['pct_discount'] = df['pct_discount'].astype(str) + "%"

In [17]:
# article_id, the article id (from the url)
df['article_id'] = df.url.str.extract("(\d{5}-[A-Z])")
df.head()

,name,price,url,price_original,price_discounted,pct_discount,article_id
0,Washed Linen Duvet Cover Set,$59.99 $129,http://www.hm.com/us/product/13472?article=13472-N,129.00,59.99,53.0%,13472-N
1,Candle in Glass Jar,$6.99 $17.99,http://www.hm.com/us/product/35079?article=35079-D,17.99,6.99,61.0%,35079-D
2,Glittery Cushion Cover,$7.99 $17.99,http://www.hm.com/us/product/72462?article=72462-A,17.99,7.99,56.0%,72462-A
3,Textured-weave Cushion Cover,$6.99 $12.99,http://www.hm.com/us/product/58926?article=58926-C,12.99,6.99,46.0%,58926-C
4,Stoneware Bowl,$17.99 $24.99,http://www.hm.com/us/product/74242?article=74242-A,24.99,17.99,28.0%,74242-A


## 1.2 Sci-Fi Authors

Open up `sci-fi.csv` to clean. Get rid of the `\n` on the title and and give me six new columns:

* `avg_rating`
* `rating_count`
* `total_score`
* `score_votes`
* `series` the series the book belongs to
* `series_no` the book in the series that it is

For series, I'm talking about e.g. `(The Hunger Games, #1)` is `series` "The Hunter Games" and `series_no` 1.

Save as **sci-fi_cleaned.csv**.

**Tip:** You don't need regex to clean the title - there's a special thing that removes whitespace from the beginning/end of strings

**Tip:** Remember that `(` and `)` are special characters

**BONUS:** When you make the `total_score` column, pay close attention to it. If you notice the problem, fix it.

**BONUS:** You don't need these columns to be numbers, but life would be better if they were. 

In [8]:
df2 = pd.read_csv("scraped/sci-fi.csv")
df2.head()

,full_rating,full_score,rank,title,url
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,\nThe Handmaid's Tale\n,/book/show/38447.The_Handmaid_s_Tale
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"\nThe Hunger Games (The Hunger Games, #1)\n",/book/show/2767052-the-hunger-games
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"\nFrankenstein, or The Modern Prometheus\n",/book/show/18490.Frankenstein_or_The_Modern_Prometheus
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"\nA Wrinkle in Time (A Wrinkle in Time Quintet, #1)\n",/book/show/18131.A_Wrinkle_in_Time
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,\nThe Left Hand of Darkness\n,/book/show/18423.The_Left_Hand_of_Darkness


In [26]:
#Get rid of the \n on the title
df2['title'] = df2['title'].str.replace(r'(\n)','')

In [9]:
df2['avg_rating'] = df2.full_rating.str.extract("(\d.*) avg rating")

In [10]:
df2['rating_count'] = df2.full_rating.str.extract(r"— (\b\d.*) ratings")

In [11]:
df2['total_score'] = df2.full_score.str.extract(r"( \d.*),")

In [12]:
df2['score_votes'] = df2.full_score.str.extract(r"(\d.*) people voted")

In [19]:
df2['series'] = df2.title.str.extract(r"\((.*),")

In [28]:
df2['series_no'] = df2.title.str.extract(r",(.*)\)")

In [31]:
df2.head()

,full_rating,full_score,rank,title,url,avg_rating,rating_count,total_score,score_votes,series,series_no
0,"4.07 avg rating — 785,502 ratings","\nscore: 28,539,\n and\n292 people voted\n \n \n",1,The Handmaid's Tale,/book/show/38447.The_Handmaid_s_Tale,4.07,"785,502","28,539",292,NaN,NaN
1,"4.34 avg rating — 5,212,935 ratings","\nscore: 27,566,\n and\n282 people voted\n \n \n",2,"The Hunger Games (The Hunger Games, #1)",/book/show/2767052-the-hunger-games,4.34,"5,212,935","27,566",282,The Hunger Games,#1
2,"3.76 avg rating — 922,308 ratings","\nscore: 20,049,\n and\n205 people voted\n \n \n",3,"Frankenstein, or The Modern Prometheus",/book/show/18490.Frankenstein_or_The_Modern_Prometheus,3.76,"922,308","20,049",205,NaN,NaN
3,"4.04 avg rating — 702,272 ratings","\nscore: 17,684,\n and\n185 people voted\n \n \n",4,"A Wrinkle in Time (A Wrinkle in Time Quintet, #1)",/book/show/18131.A_Wrinkle_in_Time,4.04,"702,272","17,684",185,A Wrinkle in Time Quintet,#1
4,"4.06 avg rating — 77,664 ratings","\nscore: 16,070,\n and\n165 people voted\n \n \n",5,The Left Hand of Darkness,/book/show/18423.The_Left_Hand_of_Darkness,4.06,"77,664","16,070",165,NaN,NaN


In [30]:
df2.to_csv("sci-fi_cleaned.csv", index=False)

## 1.3 Where you're just doing one of my former students' projects

Once upon a time my student Stefan did a project that involved some lawyer stuff. Most of the content was in PDFs, though! I converted them to text files and put them into the `pdfs` folder, and gave you code below to open up each of them and save their contents into a dataframe.

What a nice dataframe! I want you to add the following columns to it:

* `lawyer_app`, the applicant's lawyer (pro se means that they did it themselves, that's fine)
* `lawyer_gov`, the government's lawyer
* `judge`, the name of the judge
* `access`, whether the clearance is granted or denied (although you might miss a few)

Save as **court_cleaned.csv**.

**Note:** You can look at the original PDFs, they're also included.

**Note:** This uses a fun utility called `glob`, which is mostly fun because you use it as `glob.glob`. It's used to find files that match a certain filename pattern.

**BONUS:** You'll be happy once you get the judge, but make sure it doesn't have any extra punctuation on it.

**BONUS:** You can for some words using `.str.contains("blah")` and save it into new columns. Maybe `has_debt`, `has_bankruptcy`, etc.

> It's okay if it isn't perfect. Converting PDF into data rarely is! Usually you get 90% of it done with computers, then send people to enter the other 10% by hand.

In [181]:
import glob
filenames = glob.glob("pdfs/*.txt")
contents = [open(filename, encoding="utf8").read() for filename in filenames]
df3 = pd.DataFrame({'filename': filenames, 'content': contents})
pd.set_option('display.max_colwidth', 999)

In [182]:
df3.head()

,filename,content
0,pdfs/11-11916.h1.pdf.txt,"\n\nDEPARTMENT OF DEFENSE \n\nDEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\nREDACTED \n\n \n\nISCR Case No. 11-11916 \n\nMENDEZ, Francisco, Administrative Judge: \n\n \nApplicant did not mitigate security concerns raised by his exercise of foreign \ncitizenship, including the possession of a current foreign passport. He also did not \nmitigate security concerns raised by his substantial ties to Russia through which he \ncould be subjected to adverse foreign influence. Clearance is denied. \n \n\nHistory of the Case \n\nOn August 15, 2009, Applicant submitted a security clearance application (SCA). \nHe voluntarily disclosed his dual U.S.-Russian citizenship, as well as his connections \nand property interest in Russia. \n\n \nOn April 17, 2014, the Department of Defense..."
1,pdfs/12-01601.a1.pdf.txt,"KEYWORD: Guideline F\n\nDIGEST: Most of the document Applicant submitted on appeal were not previously submitted to\nthe Judge and the Board is prohibited from receiving or considering them. Adverse decision\naffirmed.\n\nCASENO: 12-01601.a1\n\nDATE: 09/23/2016\n\nIn Re:\n\n-------\n\nApplicant for Security Clearance\n\nDATE: September 23, 2016\n\nISCR Case No. 12-01601\n\n)\n)\n)\n)\n)\n)\n)\n)\n\nAPPEAL BOARD DECISION\n\nAPPEARANCES\n\nFOR GOVERNMENT\n\nJames B. Norman, Esq., Chief Department Counsel\n\nFOR APPLICANT\n\nPro se\n\n The Department of Defense (DoD) declined to grant Applicant a security clearance. On\nAugust 19, 2015, DoD issued a statement of reasons (SOR) advising Applicant of the basis for that\ndecision–security concerns raised under Guideline F (Financial Considerations) of Department of\nDefense Directive 5220.6 (Jan. 2, 1992, as amended) (Directive). Applicant requested a decision\non the written record. On June 27, 2016, after considering the record, De..."
2,pdfs/11-03073.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n\n \n\nISCR Case No. 11-03073 \n\n \n \n\n) \n) \n) \n) \n) \n \n \n\n \n \n\nAppearances \n\n______________ \n\n \nDecision \n\n______________ \n\n \n\n \n\n \n\n \n \n\nFor Government: Robert J. Kilmartin, Esq., Department Counsel \n\nFor Applicant: Mark S. Zaid, Esq. \n\nLOUGHRAN, Edward W., Administrative Judge: \n\n \nApplicant mitigated the financial considerations security concerns. Eligibility for \n\naccess to classified information is granted. \n \n\nStatement of the Case \n\nOn October 28, 2014, the Department of Defense (DOD) issued a Statement of \nReasons (SOR) to Applicant detailing security concerns under Guideline F, financial \nconsiderations. The action was taken under Executive Orde..."
3,pdfs/11-04909.h1.pdf.txt,"\n\n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n DEPARTMENT OF DEFENSE \n\n \n \n\n \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n \n \n\n \n\nISCR Case No. 11-04909 \n\n \n\nFor Government: Richard Stevens, Esq., Department Counsel \n\nFor Applicant: Pro se \n\n \n\n \nDUFFY, James F., Administrative Judge: \n\n \nApplicant mitigated \n\nconsiderations). Clearance is granted. \n\nthe security concerns under Guideline F (financial \n\nStatement of the Case \n\nOn April 5, 2015, the Department of Defense (DOD) Consolidated Adjudications \nFacility (CAF) issued Applicant a Statement of Reasons (SOR) detailing security \nconcerns under Guideline F. DOD CAF took that action under Executive Order 10865, \nSafeguarding Classified Information Within Industry, dated February 20, 1960, as \namended; DOD Directive 5220.6, Defense Industri..."
4,pdfs/11-08313.h1.pdf.txt,"\n\n DEPARTMENT OF DEFENSE \n DEFENSE OFFICE OF HEARINGS AND APPEALS \n\n \n\nISCR Case No. 11-08313 \n\n \nIn the matter of: \n \n \n \nApplicant for Security Clearance \n\n--------------- \n \n\n \n\n \n \n\n) \n) \n) \n) \n) \n \n\n \n \n\nApp

Okay, now do the work and **make those new columns!**

In [99]:
df3.content.str.extract(r"For Applicant:(.*)", re.IGNORECASE)
df3['lawyer_app'] = df3.content.str.extract(r"For Applicant:(.*)", re.IGNORECASE)

In [100]:
df3.content.str.extract(r"For Government:(.*),.*,")
df3['lawyer_gov'] = df3.content.str.extract(r"For Government:(.*),.*,")

In [107]:
 df3.content.str.extract(r"(.*), Administrative Judge")
df3['judge'] = df3.content.str.extract(r"(.*), Administrative Judge"


In [111]:
df3.content.str.extract(r"(granted|denied)")
df3['access'] = df3.content.str.extract(r"(granted|denied)")

In [110]:
df3.head()
#two of the pdfs are structured differently (1 and 11)

filename  \
0  pdfs/11-11916.h1.pdf.txt   
1  pdfs/12-01601.a1.pdf.txt   
2  pdfs/11-03073.h1.pdf.txt   
3  pdfs/11-04909.h1.pdf.txt   
4  pdfs/11-08313.h1.pdf.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [112]:
df3.to_csv("court_cleaned.csv", index=False)

# Reading books

When you're doing text work, you're legally obligated work on Jane Austen's Pride and Prejudice (at least I *think* so). Let's do some naive analysis of it!

## Read in Jane Austen's Pride and Prejudice (without moving the file!)

It's in the `data/` directory, and named `Austen_Pride.txt`.

In [189]:
contents = open("data/Austen_Pride.txt").read()

## Look at the first 500 or so characters of it 

In [190]:
contents[0:500]

' Pride and Prejudice\nby Jane Austen\nChapter 1\nIt is a truth universally acknowledged, that a single man in possession of a good fortune, must be in want of a wife.\nHowever little known the feelings or views of such a man may be on his first entering a neighbourhood, this truth is so well fixed in the minds of the surrounding families, that he is considered the rightful property of some one or other of their daughters.\n"My dear Mr. Bennet," said his lady to him one day, "have you heard that Nethe'

## Use a regular expression to find every "he" or "she" in the book. There should be about 3000 of them.

**Tip:** Do you know about **word boundaries?** `\b` means "the beginning of end of a word."

**Tip:** You might also want to use `re.IGNORECASE`. Maybe you'll need to google it? 

**Tip:** Do NOT use `re.compile`

In [191]:
#re.findall(r"\bhe\b|\bshe\b", contents, re.IGNORECASE)
he_shes = re.findall(r"\bs?he\b", contents, re.IGNORECASE)
he_shes

['he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'She',
 'she',
 'she',
 'He',
 'he',
 'she',
 'he',
 'She',
 'She',
 'she',
 'she',
 'she',
 'he',
 'he',
 'he',
 'she',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'She',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'She',
 'he',
 'he',
 'She',
 'She',
 'she',
 'she',
 'he',
 'he',
 'He',
 'he',
 'he',
 'she',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'she',
 'She',
 'she',
 'he',
 'He',
 'he',
 'she',
 'He',
 'she',
 'He',
 'he',
 'He',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he

In [212]:
len(he_shes)

3047

## Use a regular expression to find those same "he" or "she"s, but also match *the word after it*

The first four should be:

* he is
* he had
* she told
* he came

In [195]:
re.findall(r"\bs?he\b \b\w*", contents, re.IGNORECASE)

['he is',
 'he had',
 'she told',
 'he came',
 'he agreed',
 'he is',
 'he married',
 'he may',
 'he comes',
 'she ought',
 'he comes',
 'he chooses',
 'she is',
 'She was',
 'she was',
 'she fancied',
 'He had',
 'he should',
 'she had',
 'he suddenly',
 'She has',
 'She is',
 'she times',
 'she will',
 'she will',
 'he continued',
 'he wished',
 'she began',
 'she had',
 'he spoke',
 'he left',
 'he would',
 'he eluded',
 'He was',
 'he meant',
 'He had',
 'he had',
 'he saw',
 'he wore',
 'She could',
 'he could',
 'she began',
 'he might',
 'he ought',
 'he brought',
 'he had',
 'he was',
 'he was',
 'he was',
 'he was',
 'He was',
 'he would',
 'She is',
 'he looked',
 'he withdrew',
 'She is',
 'She told',
 'she had',
 'she had',
 'he was',
 'he had',
 'He had',
 'he had',
 'she entered',
 'she looked',
 'he actually',
 'she was',
 'he asked',
 'he asked',
 'he did',
 'he seemed',
 'she was',
 'he inquired',
 'she was',
 'he danced',
 'he had',
 'he would',
 'he had',
 'He is',
 

## Use capture groups to save the pronoun (he/she) as one match and the word as another

The first five should look like

```
[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed')]```

In [198]:
pronoun_verbs_list = re.findall(r"(\bs?he\b) (\b\w*)", contents, re.IGNORECASE)
pronoun_verbs_list

[('he', 'is'),
 ('he', 'had'),
 ('she', 'told'),
 ('he', 'came'),
 ('he', 'agreed'),
 ('he', 'is'),
 ('he', 'married'),
 ('he', 'may'),
 ('he', 'comes'),
 ('she', 'ought'),
 ('he', 'comes'),
 ('he', 'chooses'),
 ('she', 'is'),
 ('She', 'was'),
 ('she', 'was'),
 ('she', 'fancied'),
 ('He', 'had'),
 ('he', 'should'),
 ('she', 'had'),
 ('he', 'suddenly'),
 ('She', 'has'),
 ('She', 'is'),
 ('she', 'times'),
 ('she', 'will'),
 ('she', 'will'),
 ('he', 'continued'),
 ('he', 'wished'),
 ('she', 'began'),
 ('she', 'had'),
 ('he', 'spoke'),
 ('he', 'left'),
 ('he', 'would'),
 ('he', 'eluded'),
 ('He', 'was'),
 ('he', 'meant'),
 ('He', 'had'),
 ('he', 'had'),
 ('he', 'saw'),
 ('he', 'wore'),
 ('She', 'could'),
 ('he', 'could'),
 ('she', 'began'),
 ('he', 'might'),
 ('he', 'ought'),
 ('he', 'brought'),
 ('he', 'had'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('he', 'was'),
 ('He', 'was'),
 ('he', 'would'),
 ('She', 'is'),
 ('he', 'looked'),
 ('he', 'withdrew'),
 ('She', 'is'),
 ('She', 't

## Save those matches into a dataframe

You can give the column names with `columns=['pronoun', 'verb']`

In [270]:
columns = ['pronoun', 'verb']
pronouns_df = pd.DataFrame.from_records(pronoun_verbs_list, columns=columns)
pronouns_df.head()

,pronoun,verb
0,he,is
1,he,had
2,she,told
3,he,came
4,he,agreed


## How many times is each pronoun used?

In [209]:
pronouns_df.pronoun.value_counts()

she    1322
he     1054
She     325
He      234
Name: pronoun, dtype: int64

## Oh, wait, clean that up.

Make it only 'he' and 'she' lowercase.

It should be about 1600 'she' and 1300 'he'

In [273]:
pronouns_df.pronoun.replace("She", "she", inplace=True)
pronouns_df.pronoun.replace("He", "he", inplace=True)
pronouns_df.pronoun.value_counts()

she    1647
he     1288
Name: pronoun, dtype: int64

## What are the top 20 most common verbs?

In [234]:
pronouns_df.verb.value_counts().head(20)

was          372
had          371
could        172
is           139
would         94
has           70
did           67
will          50
might         46
should        41
felt          38
must          37
said          33
saw           32
added         31
thought       31
then          26
replied       22
continued     21
looked        21
Name: verb, dtype: int64

## What are the top 20 most common verbs for 'he', and the top 20 most common for 'she'

**Tip:** Don't use groupby, just filter. If you want to know how, though, you can also look at "value counts for different categories" on [this page](http://jonathansoma.com/lede/foundations-2017/classes/more-pandas/class-notes/)

In [238]:
pronouns_df.groupby("pronoun")['verb'].value_counts().groupby(level=0).head(20)

pronoun  verb     
he       had          166
         was          160
         is            74
         has           49
         could         40
         would         35
         did           29
         should        26
         must          24
         will          24
         might         21
         replied       14
         came          13
         may           13
         said          13
         thought       13
         does          12
         never         12
         meant         11
         then          11
she      was          212
         had          205
         could        132
         is            65
         would         59
         did           38
         felt          33
         saw           29
         will          26
         might         25
         added         23
         has           21
         said          20
         thought       18
         should        15
         then          15
         found         13
         must      

## Who cries more, men or women? Give me a percentage answer.

**Tip:** It's `cried`, because of, you know, how books are written

In [246]:
round(pronouns_df[pronouns_df['verb'] == 'cried'].pronoun.value_counts(normalize=True) * 100, 2)

she    91.67
he      8.33
Name: pronoun, dtype: float64

## How much more common is 'he' than 'she' in J.R.R. Tolkein's Fellowship of the Ring? How does that compare to Pride and Prejudice?

The book is in the same directory.

In [247]:
contents2 = open("data/Lord of the Rings - 01 - The Fellowship of the Ring - J. R. R. Tolkien - 1955.txt").read()

In [251]:
he_shes2 = re.findall(r"\bs?he\b", contents2, re.IGNORECASE)
he_shes2

['he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'He',
 'he',
 'he',
 'he',
 'he',
 'he',
 'he',
 'She',
 'he',
 'he',
 'he',
 'she',
 'he',
 'he',
 'he',
 'he

In [258]:
lotr_pronouns_df = pd.DataFrame({'pronoun': he_shes2})
lotr_pronouns_df.pronoun.replace("She", "she", inplace=True)
lotr_pronouns_df.pronoun.replace("He", "he", inplace=True)
lotr_pronouns_df.pronoun.value_counts()

he     3062
she     159
Name: pronoun, dtype: int64

In [297]:
#Lord of the Rings
round(lotr_pronouns_df.pronoun.value_counts(normalize=True) * 100, 2)

he     95.06
she     4.94
Name: pronoun, dtype: float64

In [296]:
#Pride & Prejudice
round(pronouns_df.pronoun.value_counts(normalize=True) * 100, 2)

she    56.12
he     43.88
Name: pronoun, dtype: float64

In [309]:
#Plotting via some python package I found via Google

import  plotly.plotly as py
import plotly.graph_objs as go

trace1 = go.Bar(
    x=['he', 'she'],
    y=lotr_pronouns_df.pronoun.value_counts(normalize=True)*100,
    name='Lord of the Rings'
)
trace2 = go.Bar(
    x=['he', 'she'],
    y=pronouns_df.pronoun.value_counts(normalize=True)*100,
    name='Pride & Prejudice'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='grouped-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~angelareplica/0 or inside your plot.ly account where it is named 'grouped-bar'
